In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import tweepy
import datetime
import xlsxwriter
import sys

import pandas as pd

from openpyxl import Workbook
from openpyxl import load_workbook


USERNAME = 'TheBridge_Tech'
FILENAME = 'data/TheBridge_Tech.csv'

BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAACuWcwEAAAAA630IeSOgGtCq%2BnrgmWepknqmR0E%3DfkJceYWUciKhxIjrPArQlmsMrg4YsX9UGj86RTm73XxXF5HkK8"
CONSUMER_KEY = "6PrtF01ZpIJjAgOSyavHKwn4L"
CONSUMER_SECRET = "cxMqo42nxC4qOkdL4dE7HZ3v60m7AhUzpEWyfMVPlLxuylaQCu"
ACCESS_TOKEN = "2217959742-QPjtJb9QsSSo0pNxSx8YHvxW20jO75J9l5tSGN1"
ACCESS_TOKEN_SECRET = "4sHfRZu3zlqtmqVCQ9fwUCRYHUuTVgk4KsDyznSUXKTH8"


def getClient():
    client = tweepy.Client( bearer_token = BEARER_TOKEN,
                            consumer_key = CONSUMER_KEY,
                            consumer_secret = CONSUMER_SECRET,
                            access_token = ACCESS_TOKEN,
                            access_token_secret = ACCESS_TOKEN_SECRET)
    return client

def createTweetsFile():

    # CSV header file
    # tweet_id: Id del mensaje
    # tweet_text: Cuerpo del texto del mensaje
    # created_at: Fecha del tweet
    # author_id: Id del autor
    # author_name: Nombre del autor
    # author_username: Nombre de usuario del autor
    # Métricas públicas del tweet:
    #   A.- retweet_count = retweet
    #   B.- reply_count = reply
    #   C.- like_count = like
    tweet_id = []
    tweet_text = []
    created_at = []
    author_id = []
    author_name = []
    author_username = []
    retweet_count = []
    reply_count = []
    like_count = []
    quote_count = []

    client=getClient()

    user = client.get_user(username=USERNAME)
    id = str(user.data.id)

    #YYYY-MM-DDTHH:mm:ss
    start_time = '2022-01-01T00:00:00Z'
    end_time =   '2022-05-22T23:59:59Z'

    #
    # Process data of the tweets and append them to a csv file
    #
    # How do I access includes data while using Paginator?
    # Paginator.flatten() flattens the data and iterates over each object.
    # To access includes, you’ll need to iterate through each response instead.
    for tweet in tweepy.Paginator(client.get_users_mentions, 
            id=id, 
            start_time = start_time, end_time = end_time,
            #exclude=['retweets', 'replies'],
            expansions='author_id',
            tweet_fields=['context_annotations','created_at','author_id', 'public_metrics'], 
            max_results=100).flatten(limit=1000):

        #
        # Create the DataFrame
        #
        tweet_id.append(tweet.id)  # Id del mensaje
        tweet_text.append(tweet.text) # Cuerpo del texto del mensaje
        created_at.append(tweet.created_at) # Fecha del tweet
        author_id.append(tweet.author_id) # Id del autor
        # Get the name of the author, user.name     
        user = client.get_user(id = str(tweet.author_id)) 
        author_name.append(str(user.data.name)) # Nombre del autor. The friendly name of this user, as shown on their profile.
        author_username.append(str(user.data.username)) # Nombre de usuario del autor. The Twitter handle (screen name) of this user.
        # Métricas públicas del tweet (retweet, reply, like, count)
        retweet_count.append(tweet.public_metrics.get('retweet_count'))
        reply_count.append(tweet.public_metrics.get('reply_count'))
        like_count.append(tweet.public_metrics.get('like_count'))
        quote_count.append(tweet.public_metrics.get('quote_count'))


    #header = ['tweet_id', 'tweet_text', 'created_at', 'author_id', 'author_name', 'author_username', 'retweet_count', 'reply_count', 'like_count', 'quote_count']
    df = pd.DataFrame({
        'tweet_id':tweet_id,
        'tweet_text':tweet_text,
        'created_at': created_at,
        'author_id':author_id,
        'author_name': author_name,
        'author_username': author_username, 
        'retweet_count':retweet_count,
        'reply_count':reply_count,
        'like_count':like_count,
        'quote_count':quote_count
        })
    # Create csv file
    df.to_csv(FILENAME, sep=';')


In [2]:
createTweetsFile()